In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report, precision_score,recall_score,f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
style.use('ggplot')

df = pd.read_excel('PL_2019-2020.xlsx')
df

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,DIVERTED_AREA,DISPOSAL_AREA,BOOKED_AREA,NON_BOOKED_AREA,SUPPLIED_TONNS_1,YIELD,AGE_IN_DAYS,GBL_PLANT_ID,GBL_KHATE_ID,GBL_PLOT_ID
0,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,13.0,NaN,...,0.0,0.0,4.0,0.0,187.763,46.941,426,914297,11,12
1,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,4.0,NaN,...,0.0,0.0,5.0,0.0,99.020,19.804,361,984477,10,9
2,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,5.0,NaN,...,0.0,0.0,5.0,0.0,40.679,8.136,347,985669,10,10
3,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,7.0,NaN,...,6.0,0.0,6.0,0.0,0.000,NaN,524,903118,19,26
4,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100004,VENKAPPA SHASAPPA KAMADINNI,1.0,NaN,...,0.0,0.0,5.0,0.0,57.281,11.456,387,917920,20,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,365911,KARIYAPPA HUCHAPPA HUGGI,2.0,NaN,...,0.0,2.0,0.0,2.0,0.000,NaN,65,979755,238108,327730
53421,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,369510,SAKRAPPA YALLAPPA HAVOJI,1.0,NaN,...,0.0,0.0,1.0,0.0,21.206,21.206,404,1011696,246928,337841
53422,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,1.0,NaN,...,0.0,0.0,1.0,0.0,23.849,23.849,386,1018000,250026,341175
53423,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,2.0,NaN,...,0.0,0.0,1.0,0.0,21.810,21.810,392,1018001,250026,341176


In [3]:
CV_NAMES=['86032','91010','M 265','SNK-09293','PI 00-1110','CO VSI 08005']
df1=df[(df.CV_NAME.isin (CV_NAMES)) & 
       (((df.DISPOSAL_DATE - datetime.datetime(2018,1,1)).dt.days) > 0) &
       (df.AGE_IN_DAYS>0)].copy()
df1['DISPOSAL_TIME']=((df1.DISPOSAL_DATE - datetime.datetime(2018,1,1)).dt.days)
df1

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,DISPOSAL_AREA,BOOKED_AREA,NON_BOOKED_AREA,SUPPLIED_TONNS_1,YIELD,AGE_IN_DAYS,GBL_PLANT_ID,GBL_KHATE_ID,GBL_PLOT_ID,DISPOSAL_TIME
0,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,13.0,NaN,...,0.0,4.0,0.0,187.763,46.941,426,914297,11,12,762
1,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,4.0,NaN,...,0.0,5.0,0.0,99.020,19.804,361,984477,10,9,755
2,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,5.0,NaN,...,0.0,5.0,0.0,40.679,8.136,347,985669,10,10,769
3,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,7.0,NaN,...,0.0,6.0,0.0,0.000,NaN,524,903118,19,26,704
4,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100004,VENKAPPA SHASAPPA KAMADINNI,1.0,NaN,...,0.0,5.0,0.0,57.281,11.456,387,917920,20,27,766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,365911,KARIYAPPA HUCHAPPA HUGGI,2.0,NaN,...,2.0,0.0,2.0,0.000,NaN,65,979755,238108,327730,479
53421,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,369510,SAKRAPPA YALLAPPA HAVOJI,1.0,NaN,...,0.0,1.0,0.0,21.206,21.206,404,1011696,246928,337841,749
53422,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,1.0,NaN,...,0.0,1.0,0.0,23.849,23.849,386,1018000,250026,341175,792
53423,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,2.0,NaN,...,0.0,1.0,0.0,21.810,21.810,392,1018001,250026,341176,792


In [4]:
X=df1[["CV_NAME","CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS","DISPOSAL_TIME"]].copy()
Y=df1["DIVERSION_CENTER_NAME"].copy()
X1,X2,Y1,Y2 =train_test_split(X,Y,test_size=0.1)
X1,X2,Y1,Y2

(      CV_NAME  CLUSTER_NAME  PLANT_AREA CROP_TYPE_NAME  AGE_IN_DAYS  \
 14459   86032    FACTORY-NW         1.0             R2          348   
 51803   86032   GHATAPRABHA         2.5             PL          508   
 36185   86032     MUDHOL-SE         2.0             R1           76   
 52780   91010     MAMADAPUR         1.5             PL          469   
 12112   86032    FACTORY-SW         2.0             PL          420   
 ...       ...           ...         ...            ...          ...   
 27909   86032     MUDHOL-NW         2.0             R1          341   
 48308   86032    SHIRAGUPPI         3.5             R1          227   
 35620   91010     MUDHOL-SE         4.0             R1          422   
 37722   86032     FACTORY N         4.0             R1          338   
 2506    86032  FACTORY EAST         3.0             R2          397   
 
        DISPOSAL_TIME  
 14459            723  
 51803            735  
 36185            439  
 52780            742  
 12112        

In [5]:
hgbc = HistGradientBoostingClassifier()
xgboost = XGBClassifier(use_label_encoder=False)
catboost = CatBoostClassifier()
lgbm = LGBMClassifier()
rf = RandomForestClassifier()

In [6]:
for i in Y1.index:
    Y1[i]=(1 if (Y1[i]=="GSM_HARVEST") else 0)
Y1_new=Y1.astype('int')

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

features = ["CLUSTER_NAME","CROP_TYPE_NAME","CV_NAME"]
one = OneHotEncoder(sparse=False)
transformer = ColumnTransformer([("one",
                                  one,
                                  features)],
                                  remainder="passthrough")
X1_new = pd.DataFrame(transformer.fit_transform(X1))
for i in Y2.index:
    Y2[i]=(1 if (Y2[i]=="GSM_HARVEST") else 0)
Y2_new=Y2.astype('int')
X2_new = pd.DataFrame(transformer.fit_transform(X2))
X1_new,Y1_new,X2_new,Y2_new

(        0    1    2    3    4    5    6    7    8    9   ...   22   23   24  \
 0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  0.0   
 1      0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 2      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 3      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  1.0   
 4      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 46791  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 46792  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 46793  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0   
 46794  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 46795  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  0.0   
 
         25   26   27   28   29     30

In [7]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
def cv_score(model,X,Y):
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
    print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [10]:
cv_score(catboost,X1_new,Y1_new)
cv_score(rf,X1_new,Y1_new)
cv_score(xgboost,X1_new,Y1_new)
cv_score(lgbm,X1_new,Y1_new)
cv_score(hgbc,X1_new,Y1_new)

Accuracy: 0.910 (0.005)
Accuracy: 0.901 (0.004)
Accuracy: 0.910 (0.005)
Accuracy: 0.907 (0.004)
Accuracy: 0.905 (0.005)


In [8]:
model = xgboost.fit(X1_new,Y1_new)
model.score(X2_new,Y2_new)

[21:23:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9144230769230769

In [9]:
a=model.predict_proba(X2_new)
b=model.predict(X2_new)
a,b

(array([[6.86945915e-02, 9.31305408e-01],
        [9.99973774e-01, 2.62389658e-05],
        [9.99034762e-01, 9.65233427e-04],
        ...,
        [5.30213773e-01, 4.69786227e-01],
        [2.36611843e-01, 7.63388157e-01],
        [9.87631321e-01, 1.23686595e-02]], dtype=float32),
 array([1, 0, 0, ..., 0, 1, 0]))

In [10]:
import joblib
filename = 'finalized_model.sav'
joblib.dump(model, filename)

['finalized_model.sav']

In [11]:
import json
result = X2.iloc[1].to_json(orient="split")
parsed = json.loads(result)
json.dumps(parsed, indent=4)  

'{\n    "name": 18664,\n    "index": [\n        "CV_NAME",\n        "CLUSTER_NAME",\n        "PLANT_AREA",\n        "CROP_TYPE_NAME",\n        "AGE_IN_DAYS",\n        "DISPOSAL_TIME"\n    ],\n    "data": [\n        "91010",\n        "MUDALAGI",\n        2.0,\n        "R1",\n        2,\n        348\n    ]\n}'

In [12]:
result

'{"name":18664,"index":["CV_NAME","CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS","DISPOSAL_TIME"],"data":["91010","MUDALAGI",2.0,"R1",2,348]}'

In [13]:
X2

,CV_NAME,CLUSTER_NAME,PLANT_AREA,CROP_TYPE_NAME,AGE_IN_DAYS,DISPOSAL_TIME
480,86032,FACTORY EAST,0.5,PL,459,724
18664,91010,MUDALAGI,2.0,R1,2,348
16726,91010,MUDALAGI,1.0,R2,361,713
35324,86032,MUDHOL-SE,2.0,R1,366,722
29271,91010,MUDHOL-NW,2.0,R1,102,498
...,...,...,...,...,...,...
34014,86032,MUDHOL-SW,1.5,R1,204,587
5556,86032,FACTORY-SW,1.0,PL,323,787
33155,86032,MUDHOL-SW,3.0,PL,481,706
28798,86032,MUDHOL-NW,1.5,PL,449,710


In [14]:
X2_new.iloc[1]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      1.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      1.0
22      0.0
23      0.0
24      1.0
25      0.0
26      0.0
27      0.0
28      0.0
29      2.0
30      2.0
31    348.0
Name: 1, dtype: float64

In [15]:
X.CV_NAME.value_counts()

86032           37258
91010            8860
M 265            4225
SNK-09293         915
PI 00-1110        431
CO VSI 08005      307
Name: CV_NAME, dtype: int64

In [16]:
X.CLUSTER_NAME.value_counts()

FACTORY-SW      7463
KULIGOD         5430
FACTORY EAST    5317
MUDHOL-NW       4587
RABAKAVI        3844
SHIRAGUPPI      3833
MUDALAGI        3734
MUDHOL EAST     3048
FACTORY-NW      2595
TERADAL         2579
MUDHOL-SE       2381
MUDHOL-SW       1860
BILAGI          1365
FACTORY N        851
MAMADAPUR        778
GHATAPRABHA      686
YARAGATTI        578
RABAKAVI N       542
LOKAPUR          431
KATARAKI          94
Name: CLUSTER_NAME, dtype: int64

In [17]:
X1_new.get_params

AttributeError: 'DataFrame' object has no attribute 'get_params'

In [18]:
transformer.get_params

<bound method ColumnTransformer.get_params of ColumnTransformer(remainder='passthrough',
                  transformers=[('one', OneHotEncoder(sparse=False),
                                 ['CLUSTER_NAME', 'CROP_TYPE_NAME',
                                  'CV_NAME'])])>

In [19]:
X2

,CV_NAME,CLUSTER_NAME,PLANT_AREA,CROP_TYPE_NAME,AGE_IN_DAYS,DISPOSAL_TIME
480,86032,FACTORY EAST,0.5,PL,459,724
18664,91010,MUDALAGI,2.0,R1,2,348
16726,91010,MUDALAGI,1.0,R2,361,713
35324,86032,MUDHOL-SE,2.0,R1,366,722
29271,91010,MUDHOL-NW,2.0,R1,102,498
...,...,...,...,...,...,...
34014,86032,MUDHOL-SW,1.5,R1,204,587
5556,86032,FACTORY-SW,1.0,PL,323,787
33155,86032,MUDHOL-SW,3.0,PL,481,706
28798,86032,MUDHOL-NW,1.5,PL,449,710


In [20]:
rarray=np.array(X2)
parray=model.predict_proba(X2_new)
rarray,parray

(array([['86032', 'FACTORY EAST', 0.5, 'PL', 459, 724],
        ['91010', 'MUDALAGI', 2.0, 'R1', 2, 348],
        ['91010', 'MUDALAGI', 1.0, 'R2', 361, 713],
        ...,
        ['86032', 'MUDHOL-SW', 3.0, 'PL', 481, 706],
        ['86032', 'MUDHOL-NW', 1.5, 'PL', 449, 710],
        ['86032', 'FACTORY-NW', 1.0, 'R1', 330, 732]], dtype=object),
 array([[6.86945915e-02, 9.31305408e-01],
        [9.99973774e-01, 2.62389658e-05],
        [9.99034762e-01, 9.65233427e-04],
        ...,
        [5.30213773e-01, 4.69786227e-01],
        [2.36611843e-01, 7.63388157e-01],
        [9.87631321e-01, 1.23686595e-02]], dtype=float32))

In [21]:
int(parray[1][1]*100)

0

In [23]:

import mysql.connector

db = mysql.connector.connect(host="localhost",user="Prashant",passwd="1234",database="miniproject" )
cursor = db.cursor()
for i in range (1,50):
    id=i
    user_id=int(1+(i/10))
    cv_name=rarray[i][0]
    cluster_name=rarray[i][1]
    plant_area=rarray[i][2]
    crop_type_name=rarray[i][3]
    age_in_days=rarray[i][4]
    disposal_date=rarray[i][5]
    disposal_tonns=0
    percentage=int(parray[i][1]*100)
    sql ="INSERT INTO posts( id,user_id,cv_name,cluster_name,plant_area,crop_type_name,disposal_date,disposal_tonns,age_in_days,percentage) VALUES ('%d', '%d', '%s', '%s', '%d', '%s', '%s', '%d', '%d', '%d' )" % ( id,user_id,cv_name,cluster_name,plant_area,crop_type_name,disposal_date,disposal_tonns,age_in_days,percentage)
    cursor.execute(sql)
    db.commit()